# VGG16_LargeFOV

## 1. Environment Setup

### 1.0. Check GPU


In [ ]:
! git clone https://github.com/chendonghp/deeplab-v1.git

!pip install -U cython
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

from google.colab import drive
drive.mount('/content/drive')

%cd deeplab-v1/Implementation/
! unzip /content/drive/MyDrive/data.zip -d ./dataset


In [2]:
! ls

 augmentation.py  'DeepLab_v1 - SBD, VOC 2012.ipynb'   train.py
 best.pt	   Inference.ipynb		       utils.py
 data.py	   __init__.py			      'VGG16_LargeFOV - SBD, VOC 2012.ipynb'
 dataset	   model.py


In [21]:
from data import LungImageDataset, colorDict
from torch.utils.data import DataLoader
import augmentation
import model
import train

import torch
import torchvision

# Ignore Warning
import warnings

warnings.filterwarnings(action="ignore")

In [22]:
root = r"./dataset"

train_batch_size = 30
test_batch_size = 1

train_ratio = 0.9
size = 500
train_size = int(size * train_ratio)
train_range, val_range = (0, train_size), (train_size, size)

## 2. Data Preprocessing


In [23]:
train_tf = augmentation.Mask_Aug(
    transforms=[
        augmentation.ToTensor(),
        augmentation.PILToTensor(),  # HWC to CHW
        # augmentation.Resize((256, 256)),
        augmentation.RandomCrop((224, 224)),
        augmentation.RandomHorizontalFlip(),
        # augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

val_tf = augmentation.Mask_Aug(
    transforms=[
        augmentation.ToTensor(),
        augmentation.PILToTensor(),
        # augmentation.Resize((256, 256)),
        # augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

train_dataset = LungImageDataset(root, size=train_range, transform=train_tf)
val_dataset = LungImageDataset(root, size=val_range, transform=val_tf)


train_loader = DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True
)


# train_dataset = torchvision.datasets.SBDataset(root='./', image_set='train_noval', mode='segmentation', download=False, transforms=train_tf)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)

# val_dataset = torchvision.datasets.VOCSegmentation(root='./', year='2012', image_set='val', download=False, transforms=val_tf)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
train_features, train_labels = next(iter(train_loader))

In [15]:
train_labels.shape, train_features.shape

(torch.Size([30, 1, 224, 224]), torch.Size([30, 3, 224, 224]))

## 3. Train Model


In [16]:
num_classes = len(colorDict.keys())
init_weights = True

ignore_index = 255

gpu_id = 0
print_freq = 10
epoch_print = 1

save = True
epochs = 100

lr = 0.001
momentum = 0.9
weight_decay = 0.0005
load_path = "/content/drive/MyDrive"
save_path = "/content/drive/MyDrive"
log_path = "/content/drive/MyDrive"

In [25]:
vgg16_largefov = train.VGG16_LargeFOV(
    num_classes=num_classes,
    init_weights=init_weights,
    ignore_index=ignore_index,
    gpu_id=gpu_id,
    print_freq=print_freq,
    epoch_print=epoch_print,
)

# VGG16_LargeFOV = train.VGG16_LargeFOV(num_classes=num_classes, init_weights=init_weights, ignore_index=ignore_index,
#                                       gpu_id=gpu_id, print_freq=print_freq, epoch_print=epoch_print)

In [ ]:
vgg16_largefov.train(
    train_loader,
    val_loader,
    load_path=None,
    save_path=save_path,
    log_path=log_path,
    epochs=epochs,
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay,
)
# VGG16_LargeFOV.train(train_loader, val_loader, save=save, epochs=epochs, lr=lr, momentum=momentum, weight_decay=weight_decay)